# Взять набор данных на ваше усмотрение (стихи/прозу) или что-то ещё для примера можно так же использовать прикреплённый Евгений Онегин

## Libs

In [1]:
import os

import numpy as np
import matplotlib.pyplot as plt

from nltk import word_tokenize

import tensorflow as tf
from tensorflow.keras import layers

## Get data

In [2]:
PATH_TO_FILE = '../data/evgenyi_onegin.txt'

with open(file=PATH_TO_FILE, mode='r', encoding='utf-8') as file:
    text = file.read()
    
print(text[:250])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                       


# 1. поэкспериментировать с посимвольным подходом

## Prep data

In [3]:
vocab = sorted(set(text))

idx2char = np.array(vocab)
char2idx = {u:i for i, u in enumerate(vocab)}

text_as_int = np.array([char2idx[c] for c in text])

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(101, drop_remainder=True)

In [4]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)
for input_example, target_example in  dataset.take(1):
    print('\tInput data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('\n\tTarget data:', repr(''.join(idx2char[target_example.numpy()])))

	Input data:  'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '

	Target data: 'лександр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '


In [5]:
BATCH_SIZE = 64
BUFFER_SIZE = 10_000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

## Model

In [6]:
embedding_dim = 256
rnn_units = 512

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        layers.Dense(vocab_size)
    ])
    return model

In [7]:
checkpoint_path = 'checkpoint/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1
)

In [8]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=64
)

model.compile(
    optimizer='adam', 
    loss='sparse_categorical_crossentropy'
)

try:
    model.load_weights(checkpoint_path)
    print('checkpoint loaded')
except Exception:
    print('checkpoint not found')

if True:
    history = model.fit(
        dataset, 
        epochs=5,
        callbacks=[cp_callback]
    )

checkpoint loaded
Epoch 1/5
44/44 [==============================] - 59s 1s/step - loss: 1.8788

Epoch 00001: saving model to checkpoint\cp.ckpt
Epoch 2/5
44/44 [==============================] - 53s 1s/step - loss: 1.8849

Epoch 00002: saving model to checkpoint\cp.ckpt
Epoch 3/5
44/44 [==============================] - 52s 1s/step - loss: 1.8720

Epoch 00003: saving model to checkpoint\cp.ckpt
Epoch 4/5
44/44 [==============================] - 54s 1s/step - loss: 1.8609

Epoch 00004: saving model to checkpoint\cp.ckpt
Epoch 5/5
44/44 [==============================] - 51s 1s/step - loss: 1.8436

Epoch 00005: saving model to checkpoint\cp.ckpt


# Testing model

In [9]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=1
)
model.load_weights(checkpoint_path)
model.build(tf.TensorShape([1, None]))

In [10]:
def generate_text(model, start_string, num_generate = 500, temperature=1):
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [11]:
print(generate_text(model, start_string="привет", temperature=1))

привет
      N
fXЭLю
av
d28    жанц
p8с3ьhDФрц
Но
          (Е.
   саен 
)
ъRАЖэ

ю родц

tъ8W"йэ

Цэ(
SXил сн,

 HФа
влон

)AжoP,
muс олше

  ы    ц"
N  пъчшш
ъя

     m
yМлнш
Глpн.

  SХлено
        X1ун

 ялA

   ГлъфшЛе

  IXФо

r  1z}
SI
  клац
   vs
uМл
GЗлжеее

  u"бя,

ЗDЬ2
Q-rrieq
ЬЗГчшчшл.
      7pЗА

Hw-dИ  зos0,
   {Ли

    MУншег
R4lцiАА
0Жнь.

2}
   YПалсео

'q'Ян,

L8r?  зоТ
RЖаее

8XaЗЭ2MВлwш

   к
Vут

nmOB
ъБлuyб
dБлБp)
qю

1Gmюн5Ф,
      ра

  Фа
          Гл1gЦ    k747.
  чшЯ

ORo



#### Вывод:
- обучение занимает очень много времени на ЦП
- этот подход кажется более перспективным, так как он жестко не привязан к наличию данных в датасете и модель получается более гибкой

# 2. проверить насколько изменится качество генерации текста при токенизации по словам

## Prep data

In [12]:
word_tokens = word_tokenize(text)
word_vocab = sorted(set(word_tokens))

idx2word = np.array(word_vocab)
word2idx = {w: i for i, w in enumerate(word_vocab)}

text_as_int = np.array([word2idx[w] for w in word_tokens])

In [13]:
word_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = word_dataset.batch(101, drop_remainder=True)
dataset = sequences.map(split_input_target)

for input_example, target_example in  dataset.take(1):
    print('\tInput data: ', repr(' '.join(idx2word[input_example.numpy()])))
    print('\n\tTarget data:', repr(' '.join(idx2word[target_example.numpy()])))

	Input data:  'Александр Сергеевич Пушкин Евгений Онегин Роман в стихах Не мысля гордый свет забавить , Вниманье дружбы возлюбя , Хотел бы я тебе представить Залог достойнее тебя , Достойнее души прекрасной , Святой исполненной мечты , Поэзии живой и ясной , Высоких дум и простоты ; Но так и быть - рукой пристрастной Прими собранье пестрых глав , Полусмешных , полупечальных , Простонародных , идеальных , Небрежный плод моих забав , Бессонниц , легких вдохновений , Незрелых и увядших лет , Ума холодных наблюдений И сердца горестных замет . ГЛАВА ПЕРВАЯ И жить торопится и чувствовать спешит . Кн . Вяземский'

	Target data: 'Сергеевич Пушкин Евгений Онегин Роман в стихах Не мысля гордый свет забавить , Вниманье дружбы возлюбя , Хотел бы я тебе представить Залог достойнее тебя , Достойнее души прекрасной , Святой исполненной мечты , Поэзии живой и ясной , Высоких дум и простоты ; Но так и быть - рукой пристрастной Прими собранье пестрых глав , Полусмешных , полупечальных , Простонародных ,

In [14]:
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

## Model

In [15]:
word_checkpoint_path = 'word_checkpoint/cp.ckpt'
word_checkpoint_dir = os.path.dirname(word_checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=word_checkpoint_path,
    save_weights_only=True,
    verbose=1
)

In [16]:
model = build_model(
    vocab_size=len(word_vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=64
)

model.compile(
    optimizer='adam', 
    loss='sparse_categorical_crossentropy'
)

try:
    model.load_weights(word_checkpoint_path)
    print('checkpoint loaded')
except Exception:
    print('checkpoint not found')

if True:
    history = model.fit(
        dataset, 
        epochs=5,
        callbacks=[cp_callback]
    )

checkpoint loaded
Epoch 1/5
4/4 [==============================] - 17s 4s/step - loss: 6.4165

Epoch 00001: saving model to word_checkpoint\cp.ckpt
Epoch 2/5
4/4 [==============================] - 15s 4s/step - loss: 6.6200

Epoch 00002: saving model to word_checkpoint\cp.ckpt
Epoch 3/5
4/4 [==============================] - 18s 4s/step - loss: 7.9895

Epoch 00003: saving model to word_checkpoint\cp.ckpt
Epoch 4/5
4/4 [==============================] - 16s 4s/step - loss: 8.2109

Epoch 00004: saving model to word_checkpoint\cp.ckpt
Epoch 5/5
4/4 [==============================] - 16s 4s/step - loss: 7.8681

Epoch 00005: saving model to word_checkpoint\cp.ckpt


# Testing model

In [17]:
model = build_model(
    vocab_size=len(word_vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=1
)
model.load_weights(word_checkpoint_path)
model.build(tf.TensorShape([1, None]))

In [18]:
def generate_text(model, start_string, num_generate = 500, temperature=1):
    input_eval = [word2idx[s] for s in [start_string]]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2word[predicted_id])

    return (start_string + ' '.join(text_generated))

In [19]:
print(generate_text(model, start_string="Пушкин", temperature=1, num_generate = 100))

Пушкинпомеркшею заморозил шее Жуковский начал осуждал цветет тайно Постели тайною страницы Сорочка согнала луч глупцы откуда лондонский блаженства Предтеча ласкаться окроплю живо бросают Боится клевещут ждут что-то убрана туда мельницу Благоразумной Любить Бульвары Знаком начнет шутке б отдалась тюрьмы послан дожде драки согласно всечасно сению вести пародия Хоть Вотще наводит обычный нею Замолкло эпиграмме целовать ревнивом берегов обращает волны слушает кружок Тобою благородство Затея упований шуток речкою явился вылитым поэты снимок медалью отвела костяные благоразумных высоко из Сердечной слабости псарне потонет одежд дрожало толпа Забудет ж страх тишины Харитонья деве Петушкову теша изображу мадригал обратя гусары Филипьевна грустить свечах непременно


#### Вывод:
- при обучении модели функция потерь показывает очень большой показатель
- слова в датасете уже заготовлены, так что общаться с моделью приходится исходя из содержания датасета, что делает модель менее гибкой чем при предыдущем подходе
- данный подход позволяет пропустить этап формирования слов, что ускоряет процесс генерации текста